In [1]:
%matplotlib inline

In [2]:
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model, Sequential
from keras.layers import BatchNormalization, Conv1D, Dense, Dropout, Embedding, Flatten, GRU, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

from sklearn.model_selection import train_test_split

from toxic_classifier_helper import ToxicClassifierHelper

Using TensorFlow backend.


In [3]:
VOCAB_SIZE = 25000
MAX_TEXT_LEN = 100
EMBEDDING_DIM = 128

In [ ]:
from importlib import reload
import toxic_classifier_helper
reload(toxic_classifier_helper)
from toxic_classifier_helper import ToxicClassifierHelper

In [ ]:
helper = ToxicClassifierHelper(vocab_size=VOCAB_SIZE, max_text_len=MAX_TEXT_LEN)

In [ ]:
X_train, X_test, y_train, y_test = helper.create_train_test_split(.98)

In [ ]:
model = Sequential()

embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)

model.add(embedding)
model.add(Dropout(.1))
model.add(Conv1D(filters=32, kernel_size=2, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(.1))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=2, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(GRU(32))
model.add(Dropout(.1))
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(6, activation="sigmoid"))

In [ ]:
# fasttext_weights = np.load("fasttext/fasttext_weights.npy")
# model.layers[0].set_weights(fasttext_weights)
# model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(x=np.array(X_train), y=np.array(y_train), validation_data=(X_test, y_test), epochs=1)

In [ ]:
helper.save_model(model, "test.h5")

In [ ]:
model = load_model("saved_models/test.h5")

In [ ]:
submission_input = pd.read_csv("test_csv")


In [ ]:
helper.create_submission(model, "test_submission")